# Lab session 3: GPflow

The aim of this lab session provides some practical tips for using BGPLVM, MRD on real-world datasets. We will focus on two models: 
* Bayesian Gaussian Process Latent Variable Model
* Manifold Relevance Determination

Source: Nishanth et al., IEEE TRo 2017

In [4]:
# import python modules
import GPy
import string
import numpy as np
from GPy.util import pca
from matplotlib import pyplot as plt

# call matplotlib with the inline command to make plots appear within the browser
%matplotlib inline

In [ ]:
# mrd model training
nSamples = trainData[kinectKey].shape[0]
trainList = [trainData[kinectKey], trainData[mocapKey]]
    
# initializing the latent space 
scales = []
inputX = np.zeros((nSamples,qDim))

for qD,qDV,Y in zip(qDims, qDVals, trainList):
    x,frcs = GPy.util.initialization.initialize_latent('PCA',qD, Y)
    scales.extend(frcs)
    inputX[:,qDV] = x
    
scales = np.asarray(scales)
    
# setting up the kernel
mrdKernels = []

for Y in trainList:
    mrdKernels.append(GPy.kern.RBF(qDim, variance=1., lengthscale=1./scales, ARD = True))
        
# initializing MRD model
mrdModel = GPy.models.MRD(trainList, input_dim=qDim, num_inducing=nInducing, kernel=mrdKernels, 
                          X=inputX, name='%s%d%d' % (expName,sInd,pInd+1))
print 'Setup Model!'
    
# Phase 1: Optimizaition by fixing variance parameters
var0 = mrdModel.Y0.Y.var()
var1 = mrdModel.Y1.Y.var()

mrdModel.Y0.rbf.variance.fix(var0)
mrdModel.Y1.rbf.variance.fix(var1)

mrdModel.Y0.Gaussian_noise.variance.fix(var0/SNR0)
mrdModel.Y1.Gaussian_noise.variance.fix(var1/SNR1)

mrdModel.optimize(messages=True, max_iters=initVardistIters)
    
# Phase 2: Optimize each model individually

# constrain space 0
mrdModel.Y1.constrain_fixed()
mrdModel.optimize(messages=True, max_iters=initMod0Iters)

# constrain space 1
mrdModel.Y0.constrain_fixed()
mrdModel.Y1.unconstrain_fixed()
mrdModel.Y1.rbf.variance.fix(var1)
mrdModel.Y1.Gaussian_noise.variance.fix(var1/SNR1)
mrdModel.optimize(messages=True, max_iters=initMod1Iters)
    
# Phase 3: Optimize the model without any constraints

# training without constraints
mrdModel.Y0.unconstrain_fixed()
mrdModel.Y1.unconstrain_fixed()
mrdModel.optimize(messages=True, max_iters=trainIters)
    
print 'Training Done!'
    
# plot the learned model
mrdModel.plot_scales(sharex=True,sharey=False,titles=YNames)
mrdModel.plot_latent(which_indices=[0,1])    